In [41]:
import os, sys

In [42]:
model_id = 'ContinuousSBM_bottleneck_32_firstc_10_phi_beta_cosine_betamax_2.0_betamin_0.01' #'ContinuousSBM_FFResUNet_bottleneck_32_firstc_10_phi_beta_cosine_betamax_2.0_betamin_0.01' #'ContinuousSBM_ContinuousVPSDE_I_BPROJ_bottleneck_32_firstc_10_phi_beta_cosine_betamax_0.5_betamin_0.001'
output_dir ='/mnt/home/dheurtel/ceph/04_inference/sbc/'+model_id
num_samples_per_chain = 600
num_sims_per_gpu = 400
nb_gpus = 3
noise_level_min = 0.5
noise_level_max = 1.2

In [43]:
script_template = \
"""#!/bin/bash
#SBATCH -J sbc_SigmaMN_high_{{count}}
#SBATCH --gpus=1
#SBATCH --cpus-per-gpu=16
#SBATCH --time=48:00:00
#SBATCH --partition=gpu
#SBATCH --constraint='a100-80gb|h100'
#SBATCH -o jobs/sbc_SigmaMN_high_{{count}}.log

source ~/.bashrc
source /mnt/home/dheurtel/venv/genv_DL/bin/activate
python sbc_with_MN_and_time.py --model_id {{model_id}} --num_chain={{num_sims_per_gpu}} --num_sample={{num_samples_per_chain}} --save_path={{save_path}} --noise_level_min={{noise_level_min}} --noise_level_max={{noise_level_max}} 
"""

In [44]:
keys_substitutes = {"{{model_id}}": model_id,
                    "{{num_sims_per_gpu}}": num_sims_per_gpu,
                    "{{num_samples_per_chain}}": num_samples_per_chain,
                    "{{save_path}}": os.path.join(output_dir, "tmp.pt"),
                    "{{count}}": 0,
                    "{{noise_level_min}}": noise_level_min,
                    "{{noise_level_max}}": noise_level_max}

In [45]:
for gpu_id in range(nb_gpus):
    keys_substitutes["{{count}}"] = gpu_id
    keys_substitutes["{{save_path}}"] = os.path.join(output_dir, f"sbc_sims_SigmaMN_high_{gpu_id}.pt")
    script = script_template
    for key, value in keys_substitutes.items():
        script = script.replace(key, str(value))
    with open(f'./jobs/sbc_sims_SigmaMN_nonFF_high_{gpu_id}.sh', 'w') as f:
        f.write(script)
    os.system(f'sbatch ./jobs/sbc_sims_SigmaMN_nonFF_high_{gpu_id}.sh')

Submitted batch job 3139280
Submitted batch job 3139281
Submitted batch job 3139282
